In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
!mamba install -c conda-forge rdkit

/usr/local/lib/python3.10/site-packages/conda_package_streaming/package_streaming.py:19: UserWarning: zstandard could not be imported. Running without .conda support.
  warnings.warn("zstandard could not be imported. Running without .conda support.")
/usr/local/lib/python3.10/site-packages/conda_package_handling/api.py:29: UserWarning: Install zstandard Python bindings for .conda support
  _warnings.warn("Install zstandard Python bindings for .conda support")

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║█

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import re

MSP_FILE = '/content/drive/My Drive/3DMolDrive/glycans_NIST_all.MSP'
with open(MSP_FILE, 'r') as f:
    data = f.read().split('\n\n')

#print(data[0])

def extract_info(data):
  # Extracting relevant information using regular expressions
  match_id = re.search(r'Name: (.+)', data)
  match_smiles = re.search(r'Synon: (.+)', data)
  match_precursor_type = re.search(r'Precursor_type: (.+)', data)
  match_collision_energy = re.search(r'Collision_energy: (\d+)', data)
  charge = 1  # Assuming the charge is always 1 based on the provided information
  name = re.search(r'MW: (.+)', data)

  # Printing the extracted information
  if match_id and match_smiles and match_precursor_type and match_collision_energy and name:
      id_value = match_id.group(1)
      smiles_value = match_smiles.group(1).split()[0]  # Taking the first synonym as Smiles
      precursor_type_value = match_precursor_type.group(1)
      collision_energy_value = int(match_collision_energy.group(1))
      name_value = name.group(1)

      result = {
          'ID': id_value,
          'Formula': smiles_value,
          'Precursor Type': precursor_type_value,
          'Collision Energy': collision_energy_value,
          'Charge': charge,
          'Mass': name_value
      }

      return result
  else:
      print("Unable to extract required information from the provided data.")

results_list = []
for instance in data:
  results_list.append(extract_info(instance))

unique_results_list = []
seen_smiles = set()

for result in results_list:
    if result:
      smiles = result['Formula']
    if smiles not in seen_smiles:
        unique_results_list.append(result)
        seen_smiles.add(smiles)

for glycan in unique_results_list:
  glycan['Formula'] = glycan['Formula'].replace(".alpha.", "a").replace(".beta.", "b").replace("[", "(").replace("]", ")")

# Print the unique results
for unique_result in unique_results_list:
    print(unique_result)

print(len(unique_results_list))

Unable to extract required information from the provided data.
{'ID': 'Glycan Sialylated tetraose type 1', 'Formula': 'Neu5Aca2-3Galb1-3GlcNAcb1-3Gal', 'Precursor Type': '[M-H]-', 'Collision Energy': 75, 'Charge': 1, 'Mass': '836'}
{'ID': 'Glycan Trisialyllactose', 'Formula': 'Neu5Aca2-8Neu5Acb2-8Neu5Aca2-3Galb1-4Glc', 'Precursor Type': '[M+Na-2H]-', 'Collision Energy': 27, 'Charge': 1, 'Mass': '1215'}
{'ID': "Glycan 3'-Sialyl Lewis X", 'Formula': 'NeuAca2-3Galb1-4(Fuca1-3)GlcNAc', 'Precursor Type': '[M-H]-', 'Collision Energy': 29, 'Charge': 1, 'Mass': '820'}
{'ID': 'Glycan A-Trisaccharide', 'Formula': 'GalNAca1-3(Fuca1-2)Gal', 'Precursor Type': '[M+H]+', 'Collision Energy': 2, 'Charge': 1, 'Mass': '529'}
{'ID': 'Glycan Isoglobotriaose-.beta.-N(Acetyl)-Propargyl', 'Formula': 'Gala1-3Galb1-4Glcb-Nac-Propargyl', 'Precursor Type': '[M+H]+', 'Collision Energy': 5, 'Charge': 1, 'Mass': '583'}
{'ID': 'Glycan LSTa', 'Formula': 'NeuAca2-3Galb1-3GlcNAcb1-3Galb1-4Glc', 'Precursor Type': '[M+2Na

In [ ]:
MSP_FILE = '/content/drive/My Drive/3DMolDrive/oligo_milk.MSP'
with open(MSP_FILE, 'r') as f:
    data = f.read().split('\n\n')

#print(data[0])

def extract_info(data):
    # Extracting relevant information using regular expressions
    match_id = re.search(r'Name: (.+)', data)
    match_synons = re.findall(r'Synon: (.+)', data)
    match_precursor_type = re.search(r'Precursor_type: (.+)', data)
    match_collision_energy = re.search(r'Collision_energy: (\d+)', data)
    charge = 1  # Assuming the charge is always 1 based on the provided information
    match_exact_mass = re.search(r'ExactMass: (.+)', data)

    # Choose the synonym that either has "alpha" or "beta", or contains "(some number)-(some number)"
    chosen_synon = None
    for synon in match_synons:
        if ".alpha." in synon or ".beta." in synon:
            chosen_synon = synon.replace(".alpha.", "a").replace(".beta.", "b").replace("[", "(").replace("]", ")")
            break
    if (match_id):
      if (match_id.group(1) == "Glycan 6'-Sialyl-N-acetyllactosamine"):
        chosen_synon = 'NeuAc.alpha.2-6Gal.beta.1-4GlcNac'
        chosen_synon = chosen_synon.replace(".alpha.", "a").replace(".beta.", "b").replace("[", "(").replace("]", ")")
      elif (match_id.group(1) == "Glycan 3'-Sialyl-N-acetyllactosamine"):
        chosen_synon = 'NeuAca2-3Galb1-4GlcNAc'

    if (not chosen_synon):
      for synon in match_synons:
        if re.search(r"\d-\d", synon):
            chosen_synon = synon
            chosen_synon = chosen_synon.replace("[", "(").replace("]", ")")
            chosen_synon=re.sub(r"(\D)-(\d)", r"\1\2", chosen_synon)
            chosen_synon=re.sub(r"(\d)-(\D)", r"\1\2", chosen_synon)
            chosen_synon=re.sub(r"(\D)-(\D)", r"\1\2", chosen_synon)
            #print(chosen_synon)
            break

    # Printing the extracted information
    if match_id and chosen_synon and match_precursor_type and match_collision_energy and match_exact_mass:
        id_value = match_id.group(1)
        synon_value = chosen_synon
        precursor_type_value = match_precursor_type.group(1)
        collision_energy_value = int(match_collision_energy.group(1))
        exact_mass_value = float(match_exact_mass.group(1))

        result = {
            'ID': id_value,
            'Formula': synon_value,
            'Precursor Type': precursor_type_value,
            'Collision Energy': collision_energy_value,
            'Charge': charge,
            'Mass': exact_mass_value
        }

        return result
    else:
        print("Unable to extract required information from the provided data.")

results_list = []
for instance in data:
  results_list.append(extract_info(instance))

unique_results_list_2 = []
seen_smiles = set()

for result in results_list:
    if result:
      smiles = result['ID']
    if smiles not in seen_smiles:
        unique_results_list_2.append(result)
        seen_smiles.add(smiles)

# Print the unique results
#for unique_result in unique_results_list_2:
#    print(unique_result)


unique_results_list.extend(unique_results_list_2)

for unique_result in unique_results_list:
    print(unique_result)

print(len(unique_results_list))

Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Unable to extract required information from the provide

In [ ]:
!pip install selenium
!apt-get update
!apt-get install -y chromium-browser
!apt install chromium-chromedriver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.3/448.3 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.9 MB/s eta 0:00:00
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [633 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,035 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:9 http://archive.ubuntu.com/u

In [ ]:
!pip install bs4

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 4.9 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=21c064ba2a64d3a6d4e2c0582931aa6705c4f52c16d420a6a77e3e2b6b6b6402
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup

options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
browser = webdriver.Chrome(options=options)

browser.get('https://glycotoolkit.com/Tools/GlycoGlyph/')
time.sleep(1)

for glycan in unique_results_list:
    # Input glycan
    glycan_input = browser.find_element(By.ID, 'cfg_name')
    glycan_input.clear()  # Clear any previous input
    glycan_input.send_keys(glycan['Formula'])
    time.sleep(1)

    # Click "Check Name"
    browser.find_element(By.ID, 'autoCheckName').click()

    # Handle the alert
    alert = browser.switch_to.alert
    alert.accept()

    # Wait for a moment
    time.sleep(3)

    # Read GlycoCT
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    glycoCT_element = soup.find('div', {'id': 'glycoCT'})
    glycoCT_text = glycoCT_element.get_text(separator='\n') if glycoCT_element else None
    glycan['Formula'] = glycoCT_text
    print(glycoCT_text)

# Close the browser
browser.quit()


RES
1b:o-dgal-HEX-0:0|1:aldi
2b:b-dglc-HEX-1:5
3s:n-acetyl
4b:b-dgal-HEX-1:5
5b:a-dgro-dgal-NON-2:6|1:a|2:keto|3:d
6s:n-acetyl
LIN
1:1o(3+1)2d
2:2d(2+1)3n
3:2o(3+1)4d
4:4o(3+2)5d
5:5d(5+1)6n
RES
1b:o-dglc-HEX-0:0|1:aldi
2b:b-dgal-HEX-1:5
3b:a-dgro-dgal-NON-2:6|1:a|2:keto|3:d
4s:n-acetyl
5b:b-dgro-dgal-NON-2:6|1:a|2:keto|3:d
6s:n-acetyl
7b:a-dgro-dgal-NON-2:6|1:a|2:keto|3:d
8s:n-acetyl
LIN
1:1o(4+1)2d
2:2o(3+2)3d
3:3d(5+1)4n
4:3o(8+2)5d
5:5d(5+1)6n
6:5o(8+2)7d
7:7d(5+1)8n
RES
1b:o-dglc-HEX-0:0|1:aldi
2s:n-acetyl
3b:a-lgal-HEX-1:5|6:d
4b:b-dgal-HEX-1:5
LIN
1:1d(2+1)2n
2:1o(3+1)3d
3:1o(4+1)4d
4:4o(3+2)5d
RES
1b:o-dgal-HEX-0:0|1:aldi
2b:a-lgal-HEX-1:5|6:d
3b:a-dgal-HEX-1:5
4s:n-acetyl
LIN
1:1o(2+1)2d
2:1o(3+1)3d
3:3d(2+1)4n
RES
1b:b-dglc-HEX-1:5
2b:b-dgal-HEX-1:5
3b:a-dgal-HEX-1:5
LIN
1:1o(4+1)2d
2:2o(3+1)3d
RES
1b:o-dglc-HEX-0:0|1:aldi
2b:b-dgal-HEX-1:5
3b:b-dglc-HEX-1:5
4s:n-acetyl
5b:b-dgal-HEX-1:5
LIN
1:1o(4+1)2d
2:2o(3+1)3d
3:3d(2+1)4n
4:3o(3+1)5d
5:5o(3+2)6d
RES
1b:o-dglc-HEX-0:0|1:a

In [ ]:
for glycan in unique_results_list:
  print(glycan)
  print("-----------------------------------------------------------------------")

print(len(unique_results_list))

{'ID': 'Glycan Sialylated tetraose type 1', 'Formula': 'RES\n1b:o-dgal-HEX-0:0|1:aldi\n2b:b-dglc-HEX-1:5\n3s:n-acetyl\n4b:b-dgal-HEX-1:5\n5b:a-dgro-dgal-NON-2:6|1:a|2:keto|3:d\n6s:n-acetyl\nLIN\n1:1o(3+1)2d\n2:2d(2+1)3n\n3:2o(3+1)4d\n4:4o(3+2)5d\n5:5d(5+1)6n', 'Precursor Type': '[M-H]-', 'Collision Energy': 75, 'Charge': 1, 'Mass': '836'}
-----------------------------------------------------------------------
{'ID': 'Glycan Trisialyllactose', 'Formula': 'RES\n1b:o-dglc-HEX-0:0|1:aldi\n2b:b-dgal-HEX-1:5\n3b:a-dgro-dgal-NON-2:6|1:a|2:keto|3:d\n4s:n-acetyl\n5b:b-dgro-dgal-NON-2:6|1:a|2:keto|3:d\n6s:n-acetyl\n7b:a-dgro-dgal-NON-2:6|1:a|2:keto|3:d\n8s:n-acetyl\nLIN\n1:1o(4+1)2d\n2:2o(3+2)3d\n3:3d(5+1)4n\n4:3o(8+2)5d\n5:5d(5+1)6n\n6:5o(8+2)7d\n7:7d(5+1)8n', 'Precursor Type': '[M+Na-2H]-', 'Collision Energy': 27, 'Charge': 1, 'Mass': '1215'}
-----------------------------------------------------------------------
{'ID': "Glycan 3'-Sialyl Lewis X", 'Formula': 'RES\n1b:o-dglc-HEX-0:0|1:aldi\n2s:

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-dev-shm-usage')
#options.add_argument("--window-size=1920,1080")
options.add_experimental_option("prefs", {
  "download.default_directory": "drive/MyDrive/3DMolDrive",
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})

conformations_3d = []
for glycan in unique_results_list:
  browser.quit()
  browser = webdriver.Chrome(options=options)

  browser.switch_to.default_content()

  browser.get('http://csdb.glycoscience.ru/snfgedit/snfgedit.html?expert=0&destination=structure')
  time.sleep(2)

  browser.find_element(By.XPATH, '//button[@class="btn btn-tico ti-import"]').click()
  time.sleep(1)
  browser.find_element(By.XPATH, '//button[@class="btn btn-secondary bleached" and @title="GlycoCT"]').click()

  glycan_input = browser.find_element(By.XPATH, '//textarea[@class="my-0 w-100 pre text-monospace"]')
  glycan_input.clear()  # Clear any previous input
  glycan_input.send_keys(glycan['Formula'])

  browser.find_element(By.XPATH, '//button[@class="btn btn-primary btn-block mt-3"]').click()
  try:
    browser.find_element(By.XPATH, '//button[@class="close close-white"]').click()
  except:
    print("Close button not pressed")

  time.sleep(3)

  alert = browser.switch_to.alert
  alert.accept()

  try:
      spinner_xpath = '//div[@class="spinner-border text-primary"]'
      spinner = WebDriverWait(browser, 10).until(
          EC.invisibility_of_element_located((By.XPATH, spinner_xpath))
      )

      time.sleep(8)
      # Once the spinner is invisible, try to click the button
      button_xpath = '//button[@class="button-show-3d btn btn-primary mt-2"]'
      button = browser.find_element(By.XPATH, button_xpath)
      button.click()

  except Exception as e:
      print(f"An error occurred at 3d button")
      print("CANNOT FIND CONFORMATION, STOPPING FOR:")
      print("-----------------------------------------")
      print(glycan)
      glycan['conf'] = ""
      print("-----------------------------------------")
      continue

  time.sleep(10)

  browser.execute_script("window.stop();")
  iframe = browser.find_element(By.XPATH, '//iframe[@class="jmol-iframe"]')
  browser.switch_to.frame(iframe)

  # click on link
  #link = WebDriverWait(browser, 25).until(EC.visibility_of_element_located((By.XPATH, '//a[@title="Open flat MOL file (with atom-to-residue mapping) in a separate tab"]')))
  #link.click()
  time.sleep(15)
  try:
    link = browser.find_element(By.XPATH, '//a[@title="Open flat MOL file (with atom-to-residue mapping) in a separate tab"]')
  except Exception as e:
    print(f"An error occurred: {e}")
    print("CANNOT FIND MOL FILE FOR:")
    print("-----------------------------------------")
    glycan['conf'] = ""
    print("-----------------------------------------")
    continue
  href_value = link.get_attribute('href')

  # open MOL html
  browser.switch_to.default_content()
  browser.get(href_value)

  #xpath_expression = '//pre[@style="word-wrap: break-word; white-space: pre-wrap;"]'
  #pre_element = browser.find_element(By.XPATH, xpath_expression)

  soup = BeautifulSoup(browser.page_source, 'html.parser')
  pre_element = soup.select_one('pre[style="word-wrap: break-word; white-space: pre-wrap;"]')
  pre_text = pre_element.get_text(separator='\n') if pre_element else None
  glycan['conf'] = pre_text
  print("CONFORMATIION FOUND, APPENDED")
  #print("Text Content:", pre_text)

  browser.quit()

CONFORMATIION FOUND, APPENDED
CONFORMATIION FOUND, APPENDED
An error occurred at 3d button
CANNOT FIND CONFORMATION, STOPPING FOR:
-----------------------------------------
{'ID': "Glycan 3'-Sialyl Lewis X", 'Formula': 'RES\n1b:o-dglc-HEX-0:0|1:aldi\n2s:n-acetyl\n3b:a-lgal-HEX-1:5|6:d\n4b:b-dgal-HEX-1:5\nLIN\n1:1d(2+1)2n\n2:1o(3+1)3d\n3:1o(4+1)4d\n4:4o(3+2)5d', 'Precursor Type': '[M-H]-', 'Collision Energy': 29, 'Charge': 1, 'Mass': '820'}
-----------------------------------------
CONFORMATIION FOUND, APPENDED
CONFORMATIION FOUND, APPENDED
An error occurred at 3d button
CANNOT FIND CONFORMATION, STOPPING FOR:
-----------------------------------------
{'ID': 'Glycan LSTa', 'Formula': 'RES\n1b:o-dglc-HEX-0:0|1:aldi\n2b:b-dgal-HEX-1:5\n3b:b-dglc-HEX-1:5\n4s:n-acetyl\n5b:b-dgal-HEX-1:5\nLIN\n1:1o(4+1)2d\n2:2o(3+1)3d\n3:3d(2+1)4n\n4:3o(3+1)5d\n5:5o(3+2)6d', 'Precursor Type': '[M+2Na-H]+', 'Collision Energy': 37, 'Charge': 1, 'Mass': '998'}
-----------------------------------------
An error 

In [ ]:
# remove where ['conf'] == ""
filtered_glycans = [result for result in unique_results_list if result.get('conf', '') != '']

# saving glycans (backup)
filtered_glycans_2 = filtered_glycans.copy()
filtered_glycans_3 = filtered_glycans.copy()
filtered_glycans_4 = filtered_glycans.copy()

print(len(filtered_glycans))

NameError: ignored

In [ ]:
# CONFORMATION ARRAY
def conformation_array(smiles, conf_type):
	# convert smiles to molecule
	if conf_type:
		mol = Chem.MolFromMolBlock(smiles)
		mol_from_smiles = Chem.AddHs(mol)
		AllChem.EmbedMolecule(mol_from_smiles)

	# get the x,y,z-coordinates of atoms
	try:
		conf = mol_from_smiles.GetConformer()
	except:
		return False, None, None
	xyz_arr = conf.GetPositions()
	# center the x,y,z-coordinates
	centroid = np.mean(xyz_arr, axis=0)
	xyz_arr -= centroid

	# concatenate with atom attributes
	xyz_arr = xyz_arr.tolist()
	for i, atom in enumerate(mol_from_smiles.GetAtoms()):
		xyz_arr[i] += [atom.GetDegree()]
		xyz_arr[i] += [atom.GetExplicitValence()]
		xyz_arr[i] += [atom.GetMass()/100]
		xyz_arr[i] += [atom.GetFormalCharge()]
		xyz_arr[i] += [atom.GetNumImplicitHs()]
		xyz_arr[i] += [int(atom.GetIsAromatic())]
		xyz_arr[i] += [int(atom.IsInRing())]
	xyz_arr = np.array(xyz_arr)

	# get the atom types of atoms
	atom_type = [atom.GetSymbol() for atom in mol_from_smiles.GetAtoms()]
	return True, xyz_arr, atom_type

#print(conformation_array(filtered_glycans[0]['conf'],'etkdg'))

type_counts = {}
for item in filtered_glycans:
    precursor_type = item['Precursor Type']
    type_counts[precursor_type] = type_counts.get(precursor_type, 0) + 1

# Print the counts
for precursor_type, count in type_counts.items():
    print(f'{precursor_type}: {count}')

[M-H]-: 4
[M+Na-2H]-: 1
[M+H]+: 29
[M+H-H2O]+: 1
[M+2Na]2+: 14
[M+H+K]2+: 10
[2M+Na]+: 7
[M+2H]2+: 10
[321.0]+: 1
[M-2H]2-: 6
[M+Na-3H]2-: 2
[M+2H+Na]3+: 4
[M+H+Na]2+: 3
[M+Na]+: 8
[M+NH4]+: 14
[2M-H]-: 1
[M+H2CO2-H]1-: 1
[M-H2O+H]+: 1
[M+2NH4]2+: 2
[M+NH4]-: 1
[M+H2CO2-2H]2-: 3


In [ ]:
# EXPERIMENTING WITH 3DMOLMS FUNCTIONS
from rdkit.Chem import Descriptors

# encoder (in preprocess_etkdgv3.yml)
encoder={
  'resolution': 0.2,
  'max_mz': 1500,

  'conf_type': 'etkdgv3',
  'atom_type':{'C': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'H': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'O': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    'N': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    'F': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    'S': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    'Cl': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    'P': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    'B': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    'Br': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    'I': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]},
  'max_atom_num': 300,

  'precursor_type': {
        '[M+H]+': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        '[M-H]-': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        '[M+H-H2O]+': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        '[M+Na]+': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        '[M+2H]2+': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        '[M+Na-2H]-': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        '[M+2H+Na]3+': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        '[M+H+K]2+': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        '[M+2Na-H]+': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        '[2M+H]+': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        '[M+K]+': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        '[M+2Na]+': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        '[M+H+2Na]3+': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        '[M+2Na]2+': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        '[M+H+Na]2+': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        '[M+2H+Na]3+': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        '[2M+Na]+': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        '[M-2H]2-': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        '[M+NH4]+': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        '[2M-H]-': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        '[M+NH4]-': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
    },


  'type2charge': {
    '[M+H]+': '1',       #-
    '[M-H]-': '1',
    '[M+H-H2O]+': '1',
    '[M+Na]+': '1',     #-
    '[M+2H]2+': '2',     #-
    '[M+Na-2H]-': '1',
    '[M+2H+Na]3+': '3',
    '[M+H+K]2+': '2',       #-
    '[M+2Na-H]+': '1',
    '[2M+H]+': '1',
    '[M+K]+': '1',
    '[M+2Na]+': '1',
    '[M+H+2Na]3+': '3',
    '[M+2Na]2+': '2',        #-
    '[M+H+Na]2+': '2',
    '[M+2H+Na]3+': '3',
    '[2M+Na]+': '1',         #-
    '[M-2H]2-': '2',
    '[M+NH4]+': '1',       #-
    '[2M-H]-': '1',
    '[M+NH4]-': '1'
}}

"""
encoding:
  resolution: 0.2
  max_mz: 1500

  conf_type: 'etkdgv3'
  atom_type:
    'C': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    'H': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    'O': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
    'N': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
    'F': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
    'S': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
    'Cl': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
    'P': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
    'B': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
    'Br': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
    'I': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
  max_atom_num: 300

  precursor_type:
    '[M+H]+': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    '[M-H]-': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    '[M+H-H2O]+': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    '[M+Na]+': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    '[M+2H]2+': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    '[M+Na-2H]-': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    '[M+2H+Na]3+': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    '[M+H+K]2+': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    '[M+2Na-H]+': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    '[2M+H]+': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    '[M+K]+': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    '[M+2Na]+': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    '[M+H+2Na]3+': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
    '[M+2Na]2+': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
    '[M+H+Na]2+': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
    '[M+2H+Na]3+': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
    '[2M+Na]+': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
    '[M-2H]2-': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
    '[M+NH4]+': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
    '[2M-H]-': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
    '[M+NH4]-': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

  type2charge:
    '[M+H]+': '1'
    '[M-H]-': '1'
    '[M+H-H2O]+': '1'
    '[M+Na]+': '1'
    '[M+2H]2+': '2'
    '[M+Na-2H]-': '1'
    '[M+2H+Na]3+': '3'
    '[M+H+K]2+': '2'
    '[M+2Na-H]+': '1'
    '[2M+H]+': '1'
    '[M+K]+': '1'
    '[M+2Na]+': '1'
    '[M+H+2Na]3+': '3'
    '[M+2Na]2+': '2'
    '[M+H+Na]2+': '2'
    '[M+2H+Na]3+': '3'
    '[2M+Na]+': '1'
    '[M-2H]2-': '2'
    '[M+NH4]+': '1'
    '[2M-H]-': '1'
    '[M+NH4]-': '1'
"""
# used in prediction ------------------------------------------------------------------------------

def precursor_calculator(precursor_type, mass):
  if precursor_type == '[M+H]+':
    return mass + 1.007276
  elif precursor_type == '[M+Na]+':
    return mass + 22.989218
  elif precursor_type == '[2M+H]+':
    return 2 * mass + 1.007276
  elif precursor_type == '[M-H]-':
    return mass - 1.007276
  elif precursor_type == '[M+K]+':
    return mass + 38.963158
  elif precursor_type == '[M+H+K]2+':
    return mass/2 + 19.985217
  elif precursor_type == '[M+2Na-H]+':
    return mass + 44.971160
  elif precursor_type == '[2M+Na]+':
    return 2 * mass + 22.989218
  elif precursor_type == '[M+H+2Na]3+':
    return mass/3 + 15.7661904
  elif precursor_type == '[M+2Na]2+':
    return mass/2 + 22.989218
  elif precursor_type == '[M+H+Na]2+':
    return mass/2 + + 11.998247
  elif precursor_type == '[M+2H]2+':
    return mass/2 + 1.007276
  elif precursor_type == '[M+2H+Na]3+':
    return mass/3 + 8.334590
  elif precursor_type == '[M+Na-2H]-':
    return mass + 20.974666
  elif precursor_type == '[M-2H]2-':
    return mass/2 - 1.007276
  elif precursor_type == '[M+NH4]+':
    return mass + 18.033823
  elif precursor_type == '[2M-H]-':
    return 2 * mass - 1.007276
  elif precursor_type == '[M+NH4]-':
    return mass - 18.033823
  else:
    raise ValueError('Unsupported precursor type: {}'.format(precursor_type))

def ce2nce(ce, precursor_mz, charge):
	charge_factor = {1: 1, 2: 0.9, 3: 0.85, 4: 0.8, 5: 0.75, 6: 0.75, 7: 0.75, 8: 0.75}
	return ce * 500 * charge_factor[charge] / precursor_mz

def csv2pkl_wfilter(filtered_glycans, encoder):
  '''
  This function is only used in prediction, so by default, the spectra are not contained.
  '''
  data=[]
  for idx, row in enumerate(filtered_glycans):
    # mol array
    good_conf, xyz_arr, atom_type = conformation_array(smiles=row['conf'],
                              conf_type=encoder['conf_type'])
    # There are some limitations of conformation generation methods.
    # e.g. https://github.com/rdkit/rdkit/issues/5145
    # Let's skip the unsolvable molecules.
    if not good_conf: # filter 1
      print('Can not generate correct conformation: {}'.format(row))
      continue
    if xyz_arr.shape[0] > encoder['max_atom_num']: # filter 2
      print('Atomic number ({}) exceed the limitation ({})'.format(encoder['max_atom_num'], xyz_arr.shape[0]))
      continue
    # filter 3
    rare_atom_flag = False
    rare_atom = ''
    for atom in list(set(atom_type)):
      if atom not in encoder['atom_type'].keys():
        rare_atom_flag = True
        rare_atom = atom
        break
    if rare_atom_flag:
      print('Unsupported atom type: {}'.format(rare_atom))
      continue

    if bool(re.search(r"\[\d+(\.\d+)?\]", row['Precursor Type'])):
      match = re.search(r'\](\d*\.?\d+)?', row['Precursor Type'])
      number_to_the_right = int(match.group(1)) if match and match.group(1) else 1
      row['Charge'] = number_to_the_right
    elif(row['Precursor Type'] == '[M+Na-3H]2-' or row['Precursor Type'] == '[M+2NH4]2+' or row['Precursor Type'] == '[M+H2CO2-2H]2-'):
      row['Charge'] = 2
    elif(row['Precursor Type'] == '[M+H2CO2-H]1-' or row['Precursor Type'] == '[M-H2O+H]+'):
      row['Charge'] = 1
    else:
      row['Charge'] = int(encoder['type2charge'][row['Precursor Type']])

    print("CHARGE: ")
    print(row['Charge'])

    atom_type_one_hot = np.array([encoder['atom_type'][atom] for atom in atom_type])
    assert xyz_arr.shape[0] == atom_type_one_hot.shape[0]

    mol_arr = np.concatenate([xyz_arr, atom_type_one_hot], axis=1)
    mol_arr = np.pad(mol_arr, ((0, encoder['max_atom_num']-xyz_arr.shape[0]), (0, 0)), constant_values=0)

    # env array
    if row['Precursor Type'] not in encoder['precursor_type'].keys(): # filter 4
      print('Unsupported precusor type: {}'.format(row['Precursor Type']))
      continue

    # TRY EXCEPT IGNORES VALUEERROR FROM PRECURSOR CALCULATOR
    try:
      precursor_mz = precursor_calculator(row['Precursor Type'], mass=Descriptors.MolWt(Chem.MolFromMolBlock(row['conf'])))
      print(Descriptors.MolWt(Chem.MolFromMolBlock(row['conf'])))
      print(row)
    except:
      print("precursor type not supported")
      continue
    nce = ce2nce(ce=row['Collision Energy'],
            precursor_mz=precursor_mz,
            charge=row['Charge'])
    precursor_type_one_hot = encoder['precursor_type'][row['Precursor Type']]
    env_arr = np.array([nce] + precursor_type_one_hot)

    # get mol smiles
    rdkit_mol = Chem.MolFromMolBlock(row['conf'])
    row_smiles = Chem.MolToSmiles(rdkit_mol)

    # ALSO ADDING CONF IN DATA
    data.append({'title': row['ID'], 'smiles': row_smiles, 'mol': mol_arr, 'env': env_arr, 'Mass': row['Mass']})
    print("PROCESSING DONE")
  return data

res = csv2pkl_wfilter(filtered_glycans, encoder)
print(len(res))

[06:42:27] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.


CHARGE: 
1
838.7630000000004
{'ID': 'Glycan Sialylated tetraose type 1', 'Formula': 'RES\n1b:o-dgal-HEX-0:0|1:aldi\n2b:b-dglc-HEX-1:5\n3s:n-acetyl\n4b:b-dgal-HEX-1:5\n5b:a-dgro-dgal-NON-2:6|1:a|2:keto|3:d\n6s:n-acetyl\nLIN\n1:1o(3+1)2d\n2:2d(2+1)3n\n3:2o(3+1)4d\n4:4o(3+2)5d\n5:5d(5+1)6n', 'Precursor Type': '[M-H]-', 'Collision Energy': 75, 'Charge': 1, 'Mass': '836', 'conf': 'CSDB linear = Ac(1-5)aXNeup(2-3)bDGalp(1-3)[Ac(1-2)]bDGlcpN(1-3)xDGal-ol\nMOL was produced by REStLESS, a part of CSDB (http://csdb.glycoscience.ru)\nMapping: #_x_D_Gal__-ol {7:O,8:C3,9:C2,10:O,11:C1,12:O,13:C4,14:O,15:C5,16:O,17:C6,18:O,64:H,65:H,66:H,67:H,68:H,69:H,70:H,71:H,72:H,73:H,74:H,75:H,76:H}; #3_b_D_Glc_p_N {4:N,5:C2,6:C1,19:O,20:C5,21:C6,22:O,23:C4,24:O,25:C3,26:O,61:H,62:H,63:H,77:H,78:H,79:H,80:H,81:H,82:H,83:H}; #3,2_x_X_Ac_?_ {1:C2,2:C1,3:O,58:H,59:H,60:H}; #3,3_b_D_Gal_p_ {27:C1,28:O,29:C5,30:C6,31:O,32:C4,33:O,34:C3,35:O,56:C2,57:O,84:H,85:H,86:H,87:H,88:H,89:H,90:H,91:H,110:H,111:H}; #3,3,3_a_X_

[06:42:27] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:42:27] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:42:27] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:42:27] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.


CHARGE: 
1
1218.0810000000006
{'ID': 'Glycan Trisialyllactose', 'Formula': 'RES\n1b:o-dglc-HEX-0:0|1:aldi\n2b:b-dgal-HEX-1:5\n3b:a-dgro-dgal-NON-2:6|1:a|2:keto|3:d\n4s:n-acetyl\n5b:b-dgro-dgal-NON-2:6|1:a|2:keto|3:d\n6s:n-acetyl\n7b:a-dgro-dgal-NON-2:6|1:a|2:keto|3:d\n8s:n-acetyl\nLIN\n1:1o(4+1)2d\n2:2o(3+2)3d\n3:3d(5+1)4n\n4:3o(8+2)5d\n5:5d(5+1)6n\n6:5o(8+2)7d\n7:7d(5+1)8n', 'Precursor Type': '[M+Na-2H]-', 'Collision Energy': 27, 'Charge': 1, 'Mass': '1215', 'conf': 'CSDB linear = Ac(1-5)aXNeup(2-8)[Ac(1-5)]bXNeup(2-8)[Ac(1-5)]aXNeup(2-3)bDGalp(1-4)xDGlc-ol\nMOL was produced by REStLESS, a part of CSDB (http://csdb.glycoscience.ru)\nMapping: #_x_D_Glc__-ol {15:O,16:C4,17:C3,18:O,19:C2,20:O,21:C1,22:O,23:C5,24:O,25:C6,26:O,97:H,98:H,99:H,100:H,101:H,102:H,103:H,104:H,105:H,106:H,107:H,108:H,109:H}; #4_b_D_Gal_p_ {10:O,11:C3,12:C2,13:O,14:C1,27:O,28:C5,29:C6,30:O,31:C4,32:O,93:H,94:H,95:H,96:H,110:H,111:H,112:H,113:H,114:H,115:H}; #4,3_a_X_Neu_p_ {4:N,5:C5,6:C4,7:O,8:C3,9:C2,33:C1,34:O,

[06:42:29] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:42:29] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:42:29] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:42:29] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:42:29] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:42:29] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:42:29] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:42:29] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.


...

[06:43:47] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.


Can not generate correct conformation: {'ID': 'Glycan DFLNHc', 'Formula': 'RES\n1b:o-dglc-HEX-0:0|1:aldi\n2b:b-dgal-HEX-1:5\n3b:b-dglc-HEX-1:5\n4s:n-acetyl\n5b:b-dgal-HEX-1:5\n6b:a-lgal-HEX-1:5|6:d\n7b:a-lgal-HEX-1:5|6:d\n8b:b-dglc-HEX-1:5\n9s:n-acetyl\n10b:b-dgal-HEX-1:5\nLIN\n1:1o(4+1)2d\n2:2o(3+1)3d\n3:3d(2+1)4n\n4:3o(3+1)5d\n5:5o(2+1)6d\n6:3o(4+1)7d\n7:2o(6+1)8d\n8:8d(2+1)9n\n9:8o(4+1)10d', 'Precursor Type': '[M+NH4]-', 'Collision Energy': 20, 'Charge': 1, 'Mass': 1364.49642, 'conf': 'CSDB linear = aLFucp(1-2)bDGalp(1-3)[aLFucp(1-4),Ac(1-2)]bDGlcpN(1-3)[bDGalp(1-4)[Ac(1-2)]bDGlcpN(1-6)]bDGalp(1-4)xDGlc-ol\nMOL was produced by REStLESS, a part of CSDB (http://csdb.glycoscience.ru)\nMapping: #_x_D_Glc__-ol {12:O,13:C4,14:C3,15:O,16:C2,17:O,18:C1,19:O,20:C5,21:O,22:C6,23:O,104:H,105:H,106:H,107:H,108:H,109:H,110:H,111:H,112:H,113:H,114:H,115:H,116:H}; #4_b_D_Gal_p_ {7:O,8:C6,9:C5,10:O,11:C1,24:C2,25:O,26:C3,27:O,73:C4,74:O,100:H,101:H,102:H,103:H,117:H,118:H,119:H,164:H,165:H}; #4,6_b

[06:43:48] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.


CHARGE: 
1
1367.2670000000014
{'ID': 'Glycan DFpLNH II', 'Formula': 'RES\n1b:o-dglc-HEX-0:0|1:aldi\n2b:b-dgal-HEX-1:5\n3b:b-dglc-HEX-1:5\n4s:n-acetyl\n5b:a-lgal-HEX-1:5|6:d\n6b:b-dgal-HEX-1:5\n7b:b-dglc-HEX-1:5\n8s:n-acetyl\n9b:b-dgal-HEX-1:5\n10b:a-lgal-HEX-1:5|6:d\nLIN\n1:1o(4+1)2d\n2:2o(3+1)3d\n3:3d(2+1)4n\n4:3o(3+1)5d\n5:2o(4+1)6d\n6:6o(3+1)7d\n7:7d(2+1)8n\n8:7o(3+1)9d\n9:7o(4+1)10d', 'Precursor Type': '[M+NH4]+', 'Collision Energy': 35, 'Charge': 1, 'Mass': 1364.49642, 'conf': 'CSDB linear = aLFucp(1-3)[Ac(1-2)]bDGlcpN(1-3)[bDGalp(1-3)[aLFucp(1-4),Ac(1-2)]bDGlcpN(1-3)bDGalp(1-4)]bDGalp(1-4)xDGlc-ol\nMOL was produced by REStLESS, a part of CSDB (http://csdb.glycoscience.ru)\nMapping: #_x_D_Glc__-ol {62:O,63:C4,64:C3,65:O,66:C2,67:O,68:C1,69:O,70:C5,71:O,72:C6,73:O,151:H,152:H,153:H,154:H,155:H,156:H,157:H,158:H,159:H,160:H,161:H,162:H,163:H}; #4_b_D_Gal_p_ {31:O,32:C4,33:C3,34:O,59:C2,60:O,61:C1,74:O,75:C5,76:C6,77:O,122:H,123:H,148:H,149:H,150:H,164:H,165:H,166:H,167:H}; #4,4_b_D_

[06:43:49] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:43:49] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:43:49] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:43:49] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:43:49] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:43:49] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:43:49] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:43:49] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:43:49] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol

In [ ]:
# make csv
import csv

my_list_of_dicts = filtered_glycans

# Specify the file path
file_path = '/content/drive/My Drive/3DMolDrive/input_glycan.csv'
header = my_list_of_dicts[0].keys()

# Write the list of dictionaries to the CSV file
with open(file_path, "w", newline="") as csv_file:
    csv_writer = csv.DictWriter(csv_file, fieldnames=header)

    # Write the header
    csv_writer.writeheader()

    # Write the values
    csv_writer.writerows(my_list_of_dicts)

print(f"The list of dictionaries has been written to {file_path}")

The list of dictionaries has been written to /content/drive/My Drive/3DMolDrive/input_glycan.csv


In [ ]:
import pandas as pd

def csv2pkl_wfilter(csv_path, encoder):
  '''
  This function is only used in prediction, so by default, the spectra are not contained.
  '''
  df = pd.read_csv(csv_path, encoding="unicode_escape")
  data = []
  for idx, row in df.iterrows():
    # mol array
    good_conf, xyz_arr, atom_type = conformation_array(smiles=row['conf'],
                              conf_type=encoder['conf_type'])
    # There are some limitations of conformation generation methods.
    # e.g. https://github.com/rdkit/rdkit/issues/5145
    # Let's skip the unsolvable molecules.
    if not good_conf: # filter 1
      print('Can not generate correct conformation: {}'.format(row))
      continue
    if xyz_arr.shape[0] > encoder['max_atom_num']: # filter 2
      print('Atomic number ({}) exceed the limitation ({})'.format(encoder['max_atom_num'], xyz_arr.shape[0]))
      continue
    # filter 3
    rare_atom_flag = False
    rare_atom = ''
    for atom in list(set(atom_type)):
      if atom not in encoder['atom_type'].keys():
        rare_atom_flag = True
        rare_atom = atom
        break
    if rare_atom_flag:
      print('Unsupported atom type: {}'.format(rare_atom))
      continue

    if bool(re.search(r"\[\d+(\.\d+)?\]", row['Precursor Type'])):
      match = re.search(r'\](\d*\.?\d+)?', row['Precursor Type'])
      number_to_the_right = int(match.group(1)) if match and match.group(1) else 1
      row['Charge'] = number_to_the_right
    elif(row['Precursor Type'] == '[M+Na-3H]2-' or row['Precursor Type'] == '[M+2NH4]2+' or row['Precursor Type'] == '[M+H2CO2-2H]2-'):
      row['Charge'] = 2
    elif(row['Precursor Type'] == '[M+H2CO2-H]1-' or row['Precursor Type'] == '[M-H2O+H]+'):
      row['Charge'] = 1
    else:
      row['Charge'] = int(encoder['type2charge'][row['Precursor Type']])

    print("CHARGE: ")
    print(row['Charge'])

    atom_type_one_hot = np.array([encoder['atom_type'][atom] for atom in atom_type])
    assert xyz_arr.shape[0] == atom_type_one_hot.shape[0]

    mol_arr = np.concatenate([xyz_arr, atom_type_one_hot], axis=1)
    mol_arr = np.pad(mol_arr, ((0, encoder['max_atom_num']-xyz_arr.shape[0]), (0, 0)), constant_values=0)

    # env array
    if row['Precursor Type'] not in encoder['precursor_type'].keys(): # filter 4
      print('Unsupported precusor type: {}'.format(row['Precursor Type']))
      continue

    # TRY EXCEPT IGNORES VALUEERROR FROM PRECURSOR CALCULATOR
    try:
      precursor_mz = precursor_calculator(row['Precursor Type'], mass=Descriptors.MolWt(Chem.MolFromMolBlock(row['conf'])))
      print(Descriptors.MolWt(Chem.MolFromMolBlock(row['conf'])))
      print(row)
    except:
      print("precursor type not supported")
      continue
    nce = ce2nce(ce=row['Collision Energy'],
            precursor_mz=precursor_mz,
            charge=row['Charge'])
    precursor_type_one_hot = encoder['precursor_type'][row['Precursor Type']]
    env_arr = np.array([nce] + precursor_type_one_hot)

    # get mol smiles
    rdkit_mol = Chem.MolFromMolBlock(row['conf'])
    row_smiles = Chem.MolToSmiles(rdkit_mol)

    # ALSO ADDING CONF IN DATA
    data.append({'title': row['ID'], 'smiles': row_smiles, 'mol': mol_arr, 'env': env_arr, 'Mass': row['Mass']})
    print("PROCESSING DONE")
  return data

outp = csv2pkl_wfilter('/content/drive/My Drive/3DMolDrive/input_glycan.csv', encoder)
print(len(outp))

[06:47:11] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.


CHARGE: 
1
838.7630000000004
ID                                  Glycan Sialylated tetraose type 1
Formula             RES\n1b:o-dgal-HEX-0:0|1:aldi\n2b:b-dglc-HEX-1...
Precursor Type                                                 [M-H]-
Collision Energy                                                   75
Charge                                                              1
Mass                                                            836.0
conf                CSDB linear = Ac(1-5)aXNeup(2-3)bDGalp(1-3)[Ac...
Name: 0, dtype: object
PROCESSING DONE


[06:47:11] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:47:11] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:47:11] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:47:11] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.


CHARGE: 
1
1218.0810000000006
ID                                            Glycan Trisialyllactose
Formula             RES\n1b:o-dglc-HEX-0:0|1:aldi\n2b:b-dgal-HEX-1...
Precursor Type                                             [M+Na-2H]-
Collision Energy                                                   27
Charge                                                              1
Mass                                                           1215.0
conf                CSDB linear = Ac(1-5)aXNeup(2-8)[Ac(1-5)]bXNeu...
Name: 1, dtype: object
PROCESSING DONE
CHARGE: 
1
531.5080000000003
ID                                             Glycan A-Trisaccharide
Formula             RES\n1b:o-dgal-HEX-0:0|1:aldi\n2b:a-lgal-HEX-1...
Precursor Type                                                 [M+H]+
Collision Energy                                                    2
Charge                                                              1
Mass                                                          

[06:47:13] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:47:13] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:47:13] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:47:13] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:47:13] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:47:13] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:47:13] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:47:13] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.


...

[06:48:31] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:48:31] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:48:31] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:48:31] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.


Can not generate correct conformation: ID                                                      Glycan DFLNHc
Formula             RES\n1b:o-dglc-HEX-0:0|1:aldi\n2b:b-dgal-HEX-1...
Precursor Type                                               [M+NH4]-
Collision Energy                                                   20
Charge                                                              1
Mass                                                       1364.49642
conf                CSDB linear = aLFucp(1-2)bDGalp(1-3)[aLFucp(1-...
Name: 111, dtype: object


[06:48:32] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.


Can not generate correct conformation: ID                                                   Glycan DFpLNH II
Formula             RES\n1b:o-dglc-HEX-0:0|1:aldi\n2b:b-dgal-HEX-1...
Precursor Type                                               [M+NH4]+
Collision Energy                                                   35
Charge                                                              1
Mass                                                       1364.49642
conf                CSDB linear = aLFucp(1-3)[Ac(1-2)]bDGlcpN(1-3)...
Name: 112, dtype: object
CHARGE: 
1
344.31300000000005
ID                                                     Glycan DFpLNnH
Formula             RES\n1b:o-dglc-HEX-0:0|1:aldi\n2b:b-dgal-HEX-1...
Precursor Type                                                [M+Na]+
Collision Energy                                                   35
Charge                                                              1
Mass                                                       1364.49

[06:48:32] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:48:32] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:48:32] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:48:32] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:48:32] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:48:32] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:48:32] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:48:32] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[06:48:32] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol

In [ ]:
# conformer from glycan mol
!pip install rdkit
!pip install glyles
from glyles import convert
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
import re

from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 10.1 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=a02dcf2b7e6f96ffee40547f06315ad3415ee79033f12e254b96b7058a23e189
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime
Mounted at /content/drive


In [ ]:
MSP_FILE = '/content/drive/My Drive/3DMolDrive/glycans_NIST_all.MSP'
with open(MSP_FILE, 'r') as f:
    data = f.read().split('\n\n')

#print(data[0])

type2charge={'[M+H]+': '1',
    '[M-H]-': '1',
    '[M+H-H2O]+': '1',
    '[M+Na]+': '1',
    '[M+2H]2+': '2'}

#print(type2charge.keys())

def extract_info(data):
  # Extracting relevant information using regular expressions
  match_id = re.search(r'Name: (.+)', data)
  match_smiles = re.search(r'Synon: (.+)', data)
  match_precursor_type = re.search(r'Precursor_type: (.+)', data)
  match_collision_energy = re.search(r'Collision_energy: (\d+)', data)
  charge = 1  # Assuming the charge is always 1 based on the provided information
  name = re.search(r'MW: (.+)', data)
  prec_mz = re.search(r'PrecursorMZ: (.+)', data)

  # Printing the extracted information
  if match_id and match_smiles and match_precursor_type and match_collision_energy and name and prec_mz:
      id_value = match_id.group(1)
      smiles_value = match_smiles.group(1).split()[0]  # Taking the first synonym as Smiles
      precursor_type_value = match_precursor_type.group(1)
      collision_energy_value = int(match_collision_energy.group(1))
      name_value = name.group(1)
      mz_values, intensity_values = extract_peaks_data(data)
      prec_mz = prec_mz.group(1)

      if precursor_type_value in type2charge.keys():
        result = {
            'ID': id_value,
            'Formula': smiles_value,
            'Precursor_Type': precursor_type_value,
            'Collision_Energy': collision_energy_value,
            'Mass': name_value,
            'prec_mz': prec_mz,
            'mz': mz_values,
            'intensities': intensity_values
        }
      else:
        result = {}
        print("Incompatible Precursor")

      return result
  else:
      print("Unable to extract required information from the provided data.")

def extract_peaks_data(data):
    # Find the index of the line containing "Num Peaks:"
    num_peaks_line_index = data.find("Num Peaks:")

    # Extract all lines after "Num Peaks:" containing peak data using a regular expression
    peak_lines = re.findall(r'(\d+\.\d+)\s+(\d+\.\d+)\s+".*"', data[num_peaks_line_index:])

    # Extract m/z values and intensities from the matched lines
    mz_values = [float(line[0]) for line in peak_lines]
    intensity_values = [float(line[1]) for line in peak_lines]

    # Calculate the highest intensity
    if intensity_values:
      max_intensity = max(intensity_values)

    # Calculate relative intensities
    relative_intensities = [round(intensity / max_intensity, 10) for intensity in intensity_values]

    return mz_values, relative_intensities

results_list = []
for instance in data:
  results_list.append(extract_info(instance))

unique_results_list = []
seen_smiles = set()

for result in results_list:
    if result:
      smiles = result['Formula']
    if smiles not in seen_smiles:
        unique_results_list.append(result)
        seen_smiles.add(smiles)

for glycan in unique_results_list:
  glycan['Formula'] = glycan['Formula'].replace(".alpha.", "a").replace(".beta.", "b").replace("[", "(").replace("]", ")")
  print(glycan)

print(len(unique_results_list))

# preprocessing for smiles
for entry in unique_results_list:
    # Replace ( and ) with [ and ]
    entry['Formula'] = entry['Formula'].replace('(', '[').replace(')', ']').replace("-sp21","").replace("-CH3","").replace("-Ga","").replace("-Nac-Propargyl","")

    # Put inside parenthesis every time you see (letter)(number)"-"(number)
    entry['Formula'] = re.sub(r'([a-b])(\d)-(\d)', r'(\1\2-\3)', entry['Formula'])

    # remove bad - placements (everything after it)
    entry['Formula'] = re.sub(r'(\D)-(\D).*', r'\1', entry['Formula'])
    entry['Formula'] = re.sub(r'(\D)-(\d).*', r'\1', entry['Formula'])
    entry['Formula'] = re.sub(r'(\d)-(\D).*', r'\1', entry['Formula'])
    if (entry['Formula'][-1] == "-"):
      entry['Formula'] = entry['Formula'][:-1]


Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatibl

In [ ]:
def remove_unmatched_symbols(formula):
    modified_formula = ""
    open_symbols_count = {'(': 0, '[': 0}

    for char in formula:
        if char == '(' or char == '[':
            open_symbols_count[char] += 1
        elif char == ')' or char == ']':
            symbol_type = '(' if char == ')' else '['

            if open_symbols_count[symbol_type] > 0:
                open_symbols_count[symbol_type] -= 1
            else:
                continue  # Skip unmatched closing symbol

        modified_formula += char

    # Add any remaining unmatched open symbols at the end
    modified_formula += '()' * open_symbols_count['('] + '[]' * open_symbols_count['[']

    return modified_formula

# process second NIST
MSP_FILE = '/content/drive/My Drive/3DMolDrive/oligo_milk.MSP'
with open(MSP_FILE, 'r') as f:
    data = f.read().split('\n\n')

#print(data[0])

def extract_info(data):
    # Extracting relevant information using regular expressions
    match_id = re.search(r'Name: (.+)', data)
    match_synons = re.findall(r'Synon: (.+)', data)
    match_precursor_type = re.search(r'Precursor_type: (.+)', data)
    match_collision_energy = re.search(r'Collision_energy: (\d+)', data)
    charge = 1  # Assuming the charge is always 1 based on the provided information
    match_exact_mass = re.search(r'ExactMass: (.+)', data)
    prec_mz = re.search(r'PrecursorMZ: (.+)', data)

    # Choose the synonym that either has "alpha" or "beta", or contains "(some number)-(some number)"
    chosen_synon = None
    for synon in match_synons:
        if ".alpha." in synon or ".beta." in synon:
            chosen_synon = synon.replace(".alpha.", "a").replace(".beta.", "b")
            break
    if (match_id):
      if (match_id.group(1) == "Glycan 6'-Sialyl-N-acetyllactosamine"):
        chosen_synon = 'NeuAc.alpha.2-6Gal.beta.1-4GlcNac'
        chosen_synon = chosen_synon.replace(".alpha.", "a").replace(".beta.", "b")
      elif (match_id.group(1) == "Glycan 3'-Sialyl-N-acetyllactosamine"):
        chosen_synon = 'NeuAca2-3Galb1-4GlcNAc'

    if (not chosen_synon):
      for synon in match_synons:
        if re.search(r"\d-\d", synon):
            chosen_synon = synon
            chosen_synon = chosen_synon
            chosen_synon=re.sub(r"(\D)-(\d)", r"\1\2", chosen_synon)
            chosen_synon=re.sub(r"(\d)-(\D)", r"\1\2", chosen_synon)
            chosen_synon=re.sub(r"(\D)-(\D)", r"\1\2", chosen_synon)
            #print(chosen_synon)
            break

    # Printing the extracted information
    if match_id and chosen_synon and match_precursor_type and match_collision_energy and match_exact_mass and prec_mz:
        id_value = match_id.group(1)
        synon_value = chosen_synon
        precursor_type_value = match_precursor_type.group(1)
        collision_energy_value = int(match_collision_energy.group(1))
        exact_mass_value = float(match_exact_mass.group(1))
        mz_values, intensity_values = extract_peaks_data(data)
        prec_mz = prec_mz.group(1)

        if precursor_type_value in type2charge.keys():
          result = {
              'ID': id_value,
              'Formula': synon_value,
              'Precursor_Type': precursor_type_value,
              'Collision_Energy': collision_energy_value,
              'Mass': exact_mass_value,
              'prec_mz': prec_mz,
              'mz': mz_values,
              'intensities': intensity_values
          }
        else:
          result = {}
          print("Incompatible Precursor")

        return result
    else:
        print("Unable to extract required information from the provided data.")

results_list = []
for instance in data:
  results_list.append(extract_info(instance))

unique_results_list_2 = []
seen_smiles = set()

for result in results_list:
    if result:
      smiles = result['ID']
    if smiles not in seen_smiles:
        unique_results_list_2.append(result)
        seen_smiles.add(smiles)

for entry in unique_results_list_2:
  if not bool(re.search(r'\(\D\d-\d\)', entry['Formula'])):
    # Replace ( and ) with [ and ]
    entry['Formula'] = entry['Formula'].replace('(', '[').replace(')', ']').replace("-sp21","").replace("-CH3","").replace("-Ga","").replace("-Nac-Propargyl","")

    # Put inside parenthesis every time you see (letter)(number)"-"(number)
    entry['Formula'] = re.sub(r'([a-b])(\d)-(\d)', r'(\1\2-\3)', entry['Formula'])

    # remove bad - placements (everything after it)
    entry['Formula'] = re.sub(r'(\D)-(\D).*', r'\1', entry['Formula'])
    entry['Formula'] = re.sub(r'(\D)-(\d).*', r'\1', entry['Formula'])
    entry['Formula'] = re.sub(r'(\d)-(\D).*', r'\1', entry['Formula'])
    if (entry['Formula'][-1] == "-"):
      entry['Formula'] = entry['Formula'][:-1]

    entry['Formula'] = remove_unmatched_symbols(entry['Formula'])
  entry['Formula'] = entry['Formula'].replace(" ","").replace("+","").replace('.',"")

# Print the unique results
for unique_result in unique_results_list_2:
    print(unique_result)


unique_results_list.extend(unique_results_list_2)

#for unique_result in unique_results_list:
#    print(unique_result)

print(len(unique_results_list))

# convert iupac into smiles
error_count=0
incomp_results = []
for entry in unique_results_list:
    entry['SMILES'] = convert(entry['Formula'])[0][1]
    if (entry['SMILES'] == ''):
      incomp_results.append(entry)
      error_count+=1

print(error_count)
print(f'out of {len(unique_results_list)}')

Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Incompatible Precursor
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Incompatible Precursor
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Incompatible Precursor
Incompatible Precursor
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Unable to extract required information from the provided data.
Unable to extract required information from the provided dat

line 1:4 token recognition error at: 'n'
ERROR:root:A parsing error occurred with "Chondroitin": Error message: Glycan cannot be parsed: line 1:4 token recognition error at: 'n'
line 1:1 no viable alternative at input '{2'
ERROR:root:A parsing error occurred with "2Man(a1-2)[[Man(a1-6)[Man(a1-3)]Man(a1-6)][Man(a1-2)Man(a1-3)]Man(b1-4)GlcNAc(b1-4)GlcNAc]": Error message: Glycan cannot be parsed: line 1:1 no viable alternative at input '{2'
line 1:34 no viable alternative at input '{Gal(b1-4)[[GlcNAc(b1-2)Man(a1-6)]['
ERROR:root:A parsing error occurred with "Gal(b1-4)[[GlcNAc(b1-2)Man(a1-6)][GlcNAc(b1-2)Man(a1-3)]]Man(b1-4)GlcNAc(b1-4)[Fuc(a1-6)]GlcNAc": Error message: Glycan cannot be parsed: line 1:34 no viable alternative at input '{Gal(b1-4)[[GlcNAc(b1-2)Man(a1-6)]['
line 1:42 no viable alternative at input '{Man(a1-2)[[Man(a1-6)[Man(a1-3)]Man(a1-6)]['
ERROR:root:A parsing error occurred with "Man(a1-2)[[Man(a1-6)[Man(a1-3)]Man(a1-6)][Man(a1-2)Man(a1-3)]Man(b1-4)GlcNAc(b1-4)GlcNAc]"

24
out of 125


In [ ]:
#for i in incomp_results:
#  print(i)

print(convert("Galb1-4[GlcNAcb1-2Mana1-6][GlcNAcb1-2Mana1-3]Manb1-4GlcNAcb1-4[Fuca1-6]GlcNAc"))

line 1:88 token recognition error at: 'r'
ERROR:root:A parsing error occurred with "Galb1-4[GlcNAcb1-2Mana1-6][GlcNAcb1-2Mana1-3]Manb1-4GlcNAcb1-4[Fuca1-6]GlcNAc-Nac-Propargyl": Error message: Glycan cannot be parsed: line 1:88 token recognition error at: 'r'


[('Galb1-4[GlcNAcb1-2Mana1-6][GlcNAcb1-2Mana1-3]Manb1-4GlcNAcb1-4[Fuca1-6]GlcNAc-Nac-Propargyl', '')]


In [ ]:
# Generate conformations
params = AllChem.ETKDG()   # try ETKDGv3
params.useRandomCoords=True

error_count = 0
for entry in unique_results_list:
  if (entry['SMILES']):
    try:
      mol = Chem.AddHs(Chem.MolFromSmiles(entry['SMILES']))
      AllChem.EmbedMolecule(mol, params)

      conf = mol.GetConformer()
      xyz_arr = conf.GetPositions()
      entry['xyz'] = xyz_arr

    except Exception as e:
      error_count += 1
      print(entry['Formula'])
      print(e)

print(error_count)

0


In [ ]:
# Put into MOLMS
# make csv
import csv

unique_results_list = [entry for entry in unique_results_list if entry.get('SMILES', '') != '']
print(len(unique_results_list))

my_list_of_dicts = unique_results_list

# Specify the file path
file_path = '/content/drive/My Drive/3DInput/glycan_smiles.csv'
header = my_list_of_dicts[0].keys()

# Write the list of dictionaries to the CSV file
with open(file_path, "w", newline="") as csv_file:
    csv_writer = csv.DictWriter(csv_file, fieldnames=header)

    # Write the header
    csv_writer.writeheader()

    # Write the values
    csv_writer.writerows(my_list_of_dicts)

print(f"The list of dictionaries has been written to {file_path}")


101
The list of dictionaries has been written to /content/drive/My Drive/3DInput/glycan_smiles.csv


In [ ]:
# Testing preprocessing (input into glycoglyph, compare 2d structure to 2d structure in NIST)
formula = "2Man(a1-2)[[Man(a1-6)[Man(a1-3)]Man(a1-6)][Man(a1-2)Man(a1-3)]Man(b1-4)GlcNAc(b1-4)GlcNAc]-2AA"
print(re.sub(r'(\D)-(\D).*', r'\1', formula))
print(re.sub(r'(\D)-(\d).*', r'\1', formula))
print(re.sub(r'(\d)-(\D).*', r'\1', formula))

glycans = [{'ID': "Glycan 3'-Sialyl-3-fucosyllactose", 'Formula': 'NeuNAca2-3Galb1-4(Fuca1-3)Glc', 'Precursor Type': '[M+Na]+', 'Collision Energy': 25, 'Charge': 1, 'Mass': 779.269535},
{'ID': "Glycan 3'-Galactosyllactose", 'Formula': 'Galb1-3Galb1-4Glc', 'Precursor Type': '[M+NH4]+', 'Collision Energy': 35, 'Charge': 1, 'Mass': 504.169035},
{'ID': "Glycan 2'-Fucosyllactose", 'Formula': '(Fuca1-2)Galb1-4Glc', 'Precursor Type': '[2M-H]-', 'Collision Energy': 20, 'Charge': 1, 'Mass': 488.17412},
{'ID': 'Glycan alpha-1,3-Galactose', 'Formula': 'Gal(a1-3)Gal(b1-4)GlcNAc(b1-3)Gal(b1-4)Glc', 'Precursor Type': '[M+H2CO2-H]1-', 'Collision Energy': 25, 'Charge': 1, 'Mass': 869.30123}]

print(bool(re.search(r'\([A-Z]|\)\)', '(Fuca1-2)Galb1-4Glc')))
print(re.sub(r'(\()([A-Z])', r'[\2', '(Fuca1-2)Galb1-4Glc'))
print(re.sub(r'(\))(\))', r'\1]', '(Fuc(a1-2)Gal(b1-4))Glc'))

2Man(a1-2)[[Man(a1-6)[Man(a1-3)]Man(a1-6)][Man(a1-2)Man(a1-3)]Man(b1-4)GlcNAc(b1-4)GlcNAc]-2AA
2Man(a1-2)[[Man(a1-6)[Man(a1-3)]Man(a1-6)][Man(a1-2)Man(a1-3)]Man(b1-4)GlcNAc(b1-4)GlcNAc]
2Man(a1-2)[[Man(a1-6)[Man(a1-3)]Man(a1-6)][Man(a1-2)Man(a1-3)]Man(b1-4)GlcNAc(b1-4)GlcNAc]-2AA
True
[Fuca1-2)Galb1-4Glc
(Fuc(a1-2)Gal(b1-4)]Glc


In [ ]:
import numpy as np
import re
from decimal import *
from tqdm import tqdm

from rdkit import Chem
# ignore the warning
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
from rdkit.Chem import AllChem, rdDepictor



def generate_ms(x, y, precursor_mz, resolution=1, max_mz=1500, charge=1):
	'''
	Input:  x   [float list denotes the x-coordinate of peaks]
			y   [float list denotes the y-coordinate of peaks]
			precursor_mz	[float denotes the parention]
			resolution	[float denotes the resolution of spectra]
			max_mz		[integer denotes the maxium m/z value of spectra]
			charge		[float denotes the charge of spectra]
	Return: ms	[numpy array denotes the mass spectra]
	'''
	# generate isotropic peaks (refers to Kaiyuan's codes:
	# https://github.com/lkytal/PredFull/blob/master/train_model.py)
	isotropic_peaks = []
	for delta in (0, 1, 2):
		tmp = precursor_mz + delta / charge
		isotropic_peaks.append(int(tmp // resolution))

	# prepare parameters
	precursor_mz = Decimal(str(precursor_mz))
	resolution = Decimal(str(resolution))
	max_mz = Decimal(str(max_mz))
	right_bound = int(precursor_mz // resolution) # make precursor_mz as the right bound

	# init mass spectra vector: add "0" to y data
	ms = [0] * int(max_mz // resolution)

	# convert x, y to vector
	for idx, val in enumerate(x):
		val = int(round(Decimal(str(val)) // resolution))
		if val >= right_bound: # remove precusor peak
			continue
		if val in isotropic_peaks:
			continue
		ms[val] += y[idx]

	# normalize to 0-1
	if np.max(ms) - np.min(ms) == 0:
		print('The maximum intensity and minimum intensity of this spectrum are the same!')
		print('right bound', right_bound)
		for i, j in zip(x, y):
			print(i, j)
		return False, np.array(ms)
	ms = (ms - np.min(ms)) / (np.max(ms) - np.min(ms))

	# smooth out large values
	ms = np.sqrt(np.array(ms))
	return True, ms

def ce2nce(ce, precursor_mz, charge):
	charge_factor = {1: 1, 2: 0.9, 3: 0.85, 4: 0.8, 5: 0.75, 6: 0.75, 7: 0.75, 8: 0.75}
	return ce * 500 * charge_factor[charge] / precursor_mz

def nce2ce(nce, precursor_mz, charge):
	charge_factor = {1: 1, 2: 0.9, 3: 0.85, 4: 0.8, 5: 0.75, 6: 0.75, 7: 0.75, 8: 0.75}
	return nce * precursor_mz / (500 * charge_factor[charge])

def parse_collision_energy(ce_str, precursor_mz, charge=1):
	# ratio constants for NCE
	charge_factor = {1: 1, 2: 0.9, 3: 0.85, 4: 0.8, 5: 0.75, 6: 0.75, 7: 0.75, 8: 0.75}

	ce = None
	nce = None

	# match collision energy (eV)
	matches_ev = {
		# NIST20
		r"^[\d]+[.]?[\d]*$": lambda x: float(x),
		r"^[\d]+[.]?[\d]*[ ]?eV$": lambda x: float(x.rstrip(" eV")),
		r"^[\d]+[.]?[\d]*[ ]?ev$": lambda x: float(x.rstrip(" ev")),
		r"^[\d]+[.]?[\d]*[ ]?v$": lambda x: float(x.rstrip(" v")),
		r"^[\d]+[.]?[\d]*[ ]?V$": lambda x: float(x.rstrip(" V")),
		r"^NCE=[\d]+[.]?[\d]*% [\d]+[.]?[\d]*eV$": lambda x: float(x.split()[1].rstrip("eV")),
		r"^nce=[\d]+[.]?[\d]*% [\d]+[.]?[\d]*ev$": lambda x: float(x.split()[1].rstrip("ev")),
		# MassBank
		r"^[\d]+[.]?[\d]*[ ]?V$": lambda x: float(x.rstrip(" V")),
		# r"^ramp [\d]+[.]?[\d]*-[\d]+[.]?[\d]* (ev|v)$":  lambda x: float((float(re.split(' |-', x)[1]) + float(re.split(' |-', x)[2])) /2), # j0siee: cannot process this ramp ce
		r"^[\d]+[.]?[\d]*-[\d]+[.]?[\d]*$": lambda x: float((float(x.split('-')[0]) + float(x.split('-')[1])) /2),
		r"^hcd[\d]+[.]?[\d]*$": lambda x: float(x.lstrip('hcd')),
	}
	for k, v in matches_ev.items():
		if re.match(k, ce_str):
			ce = v(ce_str)
			break
	# match collision energy (NCE)
	matches_nce = {
		# MassBank
		r"^[\d]+[.]?[\d]*[ ]?[%]? \(nominal\)$": lambda x: float(x.rstrip('% (nominal)')),
		r"^[\d]+[.]?[\d]*[ ]?nce$": lambda x: float(x.rstrip(' nce')),
		r"^[\d]+[.]?[\d]*[ ]?\(nce\)$": lambda x: float(x.rstrip(' (nce)')),
		r"^NCE=[\d]+\%$": lambda x: float(x.lstrip('NCE=').rstrip('%')),
		# casmi
		r"^[\d]+[.]?[\d]*[ ]?\(nominal\)$": lambda x: float(x.rstrip("(nominal)").rstrip(' ')),
	}
	for k, v in matches_nce.items():
		if re.match(k, ce_str):
			nce = v(ce_str) * 0.01
			break

	# unknown collision energy
	if ce_str == 'Unknown':
		ce = 40

	if nce == None and ce != None:
		nce = ce * 500 * charge_factor[charge] / precursor_mz
	elif ce == None and nce != None:
		ce = nce * precursor_mz / (500 * charge_factor[charge])
	else:
		# raise Exception('Collision energy parse error: {}'.format(ce_str))
		return None, None
	return ce, nce

def conformation_array(smiles, conf_type):
	# convert smiles to molecule
	if not smiles or smiles == '':
		return False, None, None
	params = AllChem.ETKDG()
	params.useRandomCoords=True

	if conf_type == 'etkdg':
		mol = Chem.MolFromSmiles(smiles)
		mol_from_smiles = Chem.AddHs(mol)
		AllChem.EmbedMolecule(mol_from_smiles)

	elif conf_type == 'etkdgv3':
		print(smiles)
		mol = Chem.MolFromSmiles(smiles)
		mol_from_smiles = Chem.AddHs(mol)
		AllChem.EmbedMolecule(mol_from_smiles, params)

	elif conf_type == '2d':
		mol = Chem.MolFromSmiles(smiles)
		mol_from_smiles = Chem.AddHs(mol)
		rdDepictor.Compute2DCoords(mol_from_smiles)

	elif conf_type == 'omega':
		raise ValueError('OMEGA conformation will be supported soon. ')
	else:
		raise ValueError('Unsupported conformation type. {}'.format(conf_type))

	# get the x,y,z-coordinates of atoms
	try:
		conf = mol_from_smiles.GetConformer()
	except:
		return False, None, None
	xyz_arr = conf.GetPositions()
	# center the x,y,z-coordinates
	centroid = np.mean(xyz_arr, axis=0)
	xyz_arr -= centroid

	# concatenate with atom attributes
	xyz_arr = xyz_arr.tolist()
	for i, atom in enumerate(mol_from_smiles.GetAtoms()):
		xyz_arr[i] += [atom.GetDegree()]
		xyz_arr[i] += [atom.GetExplicitValence()]
		xyz_arr[i] += [atom.GetMass()/100]
		xyz_arr[i] += [atom.GetFormalCharge()]
		xyz_arr[i] += [atom.GetNumImplicitHs()]
		xyz_arr[i] += [int(atom.GetIsAromatic())]
		xyz_arr[i] += [int(atom.IsInRing())]
	xyz_arr = np.array(xyz_arr)

	# get the atom types of atoms
	atom_type = [atom.GetSymbol() for atom in mol_from_smiles.GetAtoms()]
	return True, xyz_arr, atom_type

def precursor_calculator(precursor_type, mass):
	if precursor_type == '[M+H]+':
		return mass + 1.007276
	elif precursor_type == '[M+Na]+':
		return mass + 22.989218
	elif precursor_type == '[2M+H]+':
		return 2 * mass + 1.007276
	elif precursor_type == '[M-H]-':
		return mass - 1.007276
	elif precursor_type == '[M+H-H2O]+':
		return mass - 17.0038370665
	elif precursor_type == '[M+2H]2+':
		return mass/2 + 1.007276
	else:
		raise ValueError('Unsupported precursor type: {}'.format(precursor_type))


In [ ]:
import numpy as np
from tqdm import tqdm
from pyteomics import mgf
import pandas as pd

from rdkit import Chem
# ignore the warning
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
from rdkit.Chem import Descriptors

from .utils import conformation_array, precursor_calculator, parse_collision_energy, generate_ms, ce2nce



'''pkl for training format
[
	{'title': <str>, 'smiles': <string>,
		'mol': <numpy array>, 'env': <numpy array>, 'spec': <numpy array>},
	{'title': <str>, 'smiles': <string>,
		'mol': <numpy array>, 'env': <numpy array>, 'spec': <numpy array>},
	....
]
where 'env' is formated as np.array([normalized collision energy, one hot encoding of precursor types]).
'''

'''pkl for prediction format
[
	{'title': <str>, 'smiles': <string>,
		'mol': <numpy array>, 'env': <numpy array>},
	{'title': <str>, 'smiles': <string>,
		'mol': <numpy array>, 'env': <numpy array>},
	....
]
where 'env' is formated as np.array([normalized collision energy, one hot encoding of precursor types]).
'''

# used in training and evaluation -------------------------------------------------------------------------
def mgf2pkl(spectra, encoder):
	data = []
	for idx, spectrum in enumerate(tqdm(spectra)):
		# mol array
		good_conf, xyz_arr, atom_type = conformation_array(smiles=spectrum['params']['smiles'],
															conf_type=encoder['conf_type'])
		# There are some limitations of conformation generation methods.
		# e.g. https://github.com/rdkit/rdkit/issues/5145
		# Let's skip the unsolvable molecules.
		if not good_conf:
			continue
		atom_type_one_hot = np.array([encoder['atom_type'][atom] for atom in atom_type])
		assert xyz_arr.shape[0] == atom_type_one_hot.shape[0]

		mol_arr = np.concatenate([xyz_arr, atom_type_one_hot], axis=1)
		mol_arr = np.pad(mol_arr, ((0, encoder['max_atom_num']-xyz_arr.shape[0]), (0, 0)), constant_values=0)

		# spec array
		good_spec, spec_arr = generate_ms(x=spectrum['m/z array'],
								y=spectrum['intensity array'],
								precursor_mz=float(spectrum['params']['precursor_mz']),
								resolution=encoder['resolution'],
								max_mz=encoder['max_mz'],
								charge=int(encoder['type2charge'][spectrum['params']['precursor_type']]))
		if not good_spec: # after binning, some spectra do not have enough peaks' number
			continue

		# env array
		ce, nce = parse_collision_energy(ce_str=spectrum['params']['collision_energy'],
								precursor_mz=float(spectrum['params']['precursor_mz']),
								charge=int(encoder['type2charge'][spectrum['params']['precursor_type']]))
		if ce == None and nce == None:
			continue
		precursor_type_one_hot = encoder['precursor_type'][spectrum['params']['precursor_type']]
		env_arr = np.array([nce] + precursor_type_one_hot)

		data.append({'title': spectrum['params']['title'],
						'smiles': spectrum['params']['smiles'],
						'mol': mol_arr,
						'spec': spec_arr,
						'env': env_arr})
	return data



# used in prediction ------------------------------------------------------------------------------
def csv2pkl_wfilter(csv_path, encoder):
	'''
	This function is only used in prediction, so by default, the spectra are not contained.
	'''
	df = pd.read_csv(csv_path)
	data = []
	for idx, row in df.iterrows():
		# mol array
		good_conf, xyz_arr, atom_type = conformation_array(smiles=row['SMILES'],
															conf_type=encoder['conf_type'])
		# There are some limitations of conformation generation methods.
		# e.g. https://github.com/rdkit/rdkit/issues/5145
		# Let's skip the unsolvable molecules.
		if not good_conf: # filter 1
			print('Can not generate correct conformation: {}'.format(row['SMILES']))
			continue
		if xyz_arr.shape[0] > encoder['max_atom_num']: # filter 2
			print('Atomic number ({}) exceed the limitation ({})'.format(encoder['max_atom_num'], xyz_arr.shape[0]))
			continue
		# filter 3
		rare_atom_flag = False
		rare_atom = ''
		for atom in list(set(atom_type)):
			if atom not in encoder['atom_type'].keys():
				rare_atom_flag = True
				rare_atom = atom
				break
		if rare_atom_flag:
			print('Unsupported atom type: {}'.format(rare_atom))
			continue

		atom_type_one_hot = np.array([encoder['atom_type'][atom] for atom in atom_type])
		assert xyz_arr.shape[0] == atom_type_one_hot.shape[0]

		mol_arr = np.concatenate([xyz_arr, atom_type_one_hot], axis=1)
		mol_arr = np.pad(mol_arr, ((0, encoder['max_atom_num']-xyz_arr.shape[0]), (0, 0)), constant_values=0)

		# env array
		if row['Precursor_Type'] not in encoder['precursor_type'].keys(): # filter 4
			print('Unsupported precusor type: {}'.format(row['Precursor_Type']))
			continue
		precursor_mz = precursor_calculator(row['Precursor_Type'], mass=Descriptors.MolWt(Chem.MolFromSmiles(row['SMILES'])))
		nce = ce2nce(ce=row['Collision_Energy'],
						precursor_mz=precursor_mz,
						charge=int(encoder['type2charge'][row['Precursor_Type']]))
		precursor_type_one_hot = encoder['precursor_type'][row['Precursor_Type']]
		env_arr = np.array([nce] + precursor_type_one_hot)

		data.append({'title': row['ID'], 'smiles': row['SMILES'], 'mol': mol_arr, 'env': env_arr, 'prec_mz': row['prec_mz'], 'mz': row['mz'], 'intensities': row['intensities']})
	return data



# used in generating reference library ------------------------------------------------------------------------------
def sdf2pkl_with_cond(suppl, encoder, collision_energies, precursor_types):
	data = []
	for idx, mol in enumerate(tqdm(suppl)):
		# mol array
		good_conf, xyz_arr, atom_type = conformation_array(smiles=Chem.MolToSmiles(mol, isomericSmiles=True),
															conf_type=encoder['conf_type'])
		# There are some limitations of conformation generation methods.
		# e.g. https://github.com/rdkit/rdkit/issues/5145
		# Let's skip the unsolvable molecules.
		if not good_conf:
			continue
		atom_type_one_hot = np.array([encoder['atom_type'][atom] for atom in atom_type])
		assert xyz_arr.shape[0] == atom_type_one_hot.shape[0]

		mol_arr = np.concatenate([xyz_arr, atom_type_one_hot], axis=1)
		mol_arr = np.pad(mol_arr, ((0, encoder['max_atom_num']-xyz_arr.shape[0]), (0, 0)), constant_values=0)

		# env array
		for ce_str in collision_energies:
			for add in precursor_types:
				precursor_mz = precursor_calculator(add, Descriptors.ExactMolWt(mol))
				ce, nce = parse_collision_energy(ce_str=ce_str,
										precursor_mz=precursor_mz,
										charge=int(encoder['type2charge'][add]))
				if ce == None and nce == None:
					continue
				precursor_type_one_hot = encoder['precursor_type'][add]
				env_arr = np.array([nce] + precursor_type_one_hot)

				data.append({'title': mol.GetProp('DATABASE_ID')+'_'+ce_str+'_'+str(add),
							'smiles': Chem.MolToSmiles(mol, isomericSmiles=True), 'mol': mol_arr, 'env': env_arr})
	return data



In [ ]:
import os
import argparse
import numpy as np
import pandas as pd
from tqdm import tqdm
import yaml
import pickle
from pyteomics import mgf

import torch
from torch.utils.data import DataLoader

from rdkit import Chem
# ignore the warning
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
from rdkit.Chem import Descriptors

from molmspack.molnet import MolNet_MS
from molmspack.dataset import Mol_Dataset
from molmspack.data_utils import csv2pkl_wfilter, nce2ce, precursor_calculator
from molmspack.data_utils import filter_spec, mgf2pkl
from molmspack.data_utils import generate_ms

global batch_size
batch_size = 1



def spec_convert(spec, resolution):
	x = []
	y = []
	for i, j in enumerate(spec):
		if j != 0:
			x.append(str(i*resolution))
			y.append(str(j))
	return {'m/z': ','.join(x), 'intensity': ','.join(y)}

def pred_step(model, device, loader, batch_size, num_points):
	model.eval()
	id_list = []
	pred_list = []
	with tqdm(total=len(loader)) as bar:
		for step, batch in enumerate(loader):
			ids, x, env = batch
			x = x.to(device=device, dtype=torch.float)
			x = x.permute(0, 2, 1)
			env = env.to(device=device, dtype=torch.float)
			idx_base = torch.arange(0, batch_size, device=device).view(-1, 1, 1) * num_points

			with torch.no_grad():
				pred = model(x, env, idx_base)
				pred = pred / torch.max(pred) # normalize the output

			bar.set_description('Eval')
			bar.update(1)

			# recover sqrt spectra to original spectra
			pred = torch.pow(pred, 2)
			# post process
			pred = pred.detach().cpu().apply_(lambda x: x if x > 0.01 else 0)

			id_list += ids
			pred_list.append(pred)

	pred_list = torch.cat(pred_list, dim = 0)
	return id_list, pred_list

def init_random_seed(seed):
	np.random.seed(seed)
	torch.manual_seed(seed)
	torch.cuda.manual_seed(seed)
	return



if __name__ == "__main__":
	parser = argparse.ArgumentParser(description='Molecular Mass Spectra Prediction (Pred)')
	parser.add_argument('--test_data', type=str, default='',
						help='path to test data (csv/mgf/pkl)')
	parser.add_argument('--save_pkl', action='store_true',
						help='save converted pkl file')
	parser.add_argument('--model_config_path', type=str, default='./config/molnet.yml',
						help='path to model and training configuration')
	parser.add_argument('--data_config_path', type=str, default='./config/preprocess_etkdg.yml',
						help='path to data configuration')
	parser.add_argument('--resume_path', type=str, default='',
						help='path to pretrained model')
	parser.add_argument('--result_path', type=str, default='',
						help='path to saving results')

	parser.add_argument('--seed', type=int, default=42,
						help='seed for random functions')
	parser.add_argument('--device', type=int, default=0,
						help='which gpu to use if any')
	parser.add_argument('--no_cuda', action='store_true',
						help='enables CUDA training')
	args = parser.parse_args()

	init_random_seed(args.seed)
	with open(args.model_config_path, 'r') as f:
		config = yaml.load(f, Loader=yaml.FullLoader)
	print('Load the model & training configuration from {}'.format(args.model_config_path))
	with open(args.data_config_path, 'r') as f:
		data_config = yaml.load(f, Loader=yaml.FullLoader)
	print('Load the data configuration from {}'.format(args.data_config_path))

	# 1. Data
	test_format = args.test_data.split('.')[-1]
	if test_format == 'csv': # convert csv file into pkl
		pkl_dict = csv2pkl_wfilter(args.test_data, data_config['encoding'])

	elif test_format == 'mgf': # convert mgf file into pkl
		origin_spectra = mgf.read(args.test_data)

		print('Filter spectra...')
		filter_spectra, _ = filter_spec(origin_spectra,
										config['all'],
										type2charge=config['encoding']['type2charge'])
		pkl_dict = mgf2pkl(filter_spectra, config['encoding'])

	elif test_format == 'pkl': # load pkl directly
		with open(args.test_data, 'rb') as file:
			pkl_dict = pickle.load(file)

	else:
		raise ValueError('Unsupported format: {}'.format(test_format))
	print('Load {} data from {}'.format(len(pkl_dict), args.test_data))

	# same the pkl, so do not need to convert it again next time
	pkl_path = args.test_data.replace('.'+test_format, '.pkl')
	if args.save_pkl:
		if not os.path.exists(pkl_path):
			raise OSError('The pkl file exists. Do not need to save it again. ')

		with open(pkl_path, 'wb') as f:
			pickle.dump(pkl_dict, f)
			print('Save converted pkl file to {}'.format(pkl_path))

	valid_set = Mol_Dataset(pkl_dict)
	valid_loader = DataLoader(
					valid_set,
					batch_size=batch_size,
					shuffle=False,
					num_workers=config['train']['num_workers'],
					drop_last=True)

	# 2. Model
	device = torch.device("cuda:" + str(args.device)) if torch.cuda.is_available() and not args.no_cuda else torch.device("cpu")
	print(f'Device: {device}')

	model = MolNet_MS(config['model']).to(device)
	num_params = sum(p.numel() for p in model.parameters())
	print(f'{str(model)} #Params: {num_params}')

	# 3. Evaluation
	print("Load the checkpoints...")
	model.load_state_dict(torch.load(args.resume_path, map_location=device)['model_state_dict'])
	best_valid_acc = torch.load(args.resume_path, map_location=device)['best_val_acc']

	id_list, pred_list = pred_step(model, device, valid_loader,
									batch_size=batch_size, num_points=config['model']['max_atom_num'])
	pred_list = [spec_convert(spec, config['model']['resolution']) for spec in pred_list.tolist()]
	pred_mz = [pred['m/z'] for pred in pred_list]
	pred_intensity = [pred['intensity'] for pred in pred_list]

	# 4. Output the results
	result_dir = "".join(args.result_path.split('/')[:-1])
	os.makedirs(result_dir, exist_ok = True)

	# convert 'env' in pkl_dict back to experimental conditions
	decoding_precursor_type = {','.join(map(str, v)): k for k, v in data_config['encoding']['precursor_type'].items()}
	ce_list = []
	add_list = []
	smiles_list = []
	mz_list = []
	inten_list = []
	prec_mz_list = []
	for d in pkl_dict:
		mz_list.append(d['mz'])
		inten_list.append(d['intensities'])
		precursor_type = decoding_precursor_type[','.join(map(str, map(int, d['env'][1:])))]
		smiles = d['smiles']
		mass = Descriptors.ExactMolWt(Chem.MolFromSmiles(smiles))
		precurosr_mz = precursor_calculator(precursor_type, mass)
		prec_mz_list.append(precurosr_mz)
		ce_list.append(nce2ce(d['env'][0], precurosr_mz, int(data_config['encoding']['type2charge'][precursor_type])))
		add_list.append(precursor_type)
		smiles_list.append(smiles)


	def convert_into_list(str_values): # convert str into list
		str_list = str_values.split(',')

		# Convert each substring to a float
		float_list = [float(value) for value in str_list]

		return float_list

	cos_sim = []
	error_cnt = 0
	print(len(smiles_list))
	for i in range(len(smiles_list)):
		pred_ms_bool,exp_ms_bool = False,False
		if (not (pred_mz[i] == '') and not (pred_intensity[i] == '') and not (mz_list[i][1:-1] == '') and not (inten_list[i][1:-1] == '')):
			pred_ms_bool, pred_ms = generate_ms(convert_into_list(pred_mz[i]), convert_into_list(pred_intensity[i]), prec_mz_list[i], resolution=data_config['encoding']['resolution'],
									max_mz=data_config['encoding']['max_mz'],
									charge=int(data_config['encoding']['type2charge'][add_list[i]]))
			print(len(pred_mz[i]))
			print(len(pred_intensity[i]))
			print(len(mz_list[i]))
			print(len(inten_list[i]))
			print()
			if (len(inten_list[i]) != len(mz_list[i])):
				exp_ms_bool, exp_ms = generate_ms(convert_into_list(mz_list[i][1:-1]), convert_into_list(inten_list[i][1:-1]), prec_mz_list[i], resolution=data_config['encoding']['resolution'],
										max_mz=data_config['encoding']['max_mz'],
										charge=int(data_config['encoding']['type2charge'][add_list[i]]))
			else:
				error_cnt += 1
				continue
		if (pred_ms_bool and exp_ms_bool):
			sim = np.dot(pred_ms, exp_ms)/(np.linalg.norm(pred_ms)*np.linalg.norm(exp_ms))
			print(sim)
			cos_sim.append(sim)
	print(sum(cos_sim)/len(cos_sim))
	print(cos_sim)
	print(f'Not including {error_cnt} number of glycans)')




	# save the final results
	res_df = pd.DataFrame({'ID': id_list, 'SMILES': smiles_list,
							'Collision Energy': ce_list, 'Precursor Type': add_list,
							'prec_mz': prec_mz_list, 'Pred M/Z': pred_mz, 'Pred Intensity': pred_intensity,
                        'M/Z': mz_list, 'Intensity': inten_list})
	if args.result_path[-3:] == 'csv': # save results to csv file
		res_df.to_csv(args.result_path, sep='\t')
	elif args.result_path[-3:] == 'mgf': # save results to mgf file
		spectra = []
		prefix = 'pred'
		for idx, row in res_df.iterrows():
			spectrum = {
				'params': {
					'title': row['ID'],
					'mslevel': '2',
					'organism': '3DMolMS_v1.1',
					'spectrumid': prefix+'_'+str(idx),
					'smiles': row['SMILES'],
					'collision_energy': row['Collision Energy'],
					'precursor_type': row['Precursor Type'],
				},
				'm/z array': np.array([float(i) for i in row['Pred M/Z'].split(',')]),
				'intensity array': np.array([float(i)*1000 for i in row['Pred Intensity'].split(',')])
			}
			spectra.append(spectrum)
		mgf.write(spectra, args.result_path, file_mode="w", write_charges=False)
	else:
		raise Exception("Not implemented output format. Please choose `.csv` or `.mgf`.")

	print('Save the test results to {}'.format(args.result_path))